In [7]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 
from jinja2 import Environment, FileSystemLoader
import pdfkit


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = ({'host':'192.168.1.13' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},)

datalist=[]
 
for company in datacompany:
    msg_sql= """

                    select  
                                   a.cardcode ,
                                   b.cardname ,
                                   b.CntctPrsn ,
                                   b.LicTradNum npwp,
                                   b.U_AR_Person ,
                                   b.balance ,
                                   b.U_locktimeout,
                                   a.DocNum ,
                                   convert(varchar,a.docdate,23) docdate,
                                   a.NumAtCard so_number,
                                   a.U_IDU_FPajak ,
                                   a.U_Kw_No,
                                   a.DocTotal ,
                                   a.DocTotal - a.paidsys invoice_balance 


                    from OINV A 
                        inner join ocrd b on a.cardcode = b.cardcode 
                    where a.canceled='N'
                        and a.doctotal - a.paidsum <>0
                        and a.DocStatus ='O' 
                        and convert(varchar,DATEADD(month,-1 * isnull(b.u_locktimeout,2) ,convert(varchar,getdate(),112)) ,112)>= convert(varchar,a.docdate,112) 
                        --and b.u_lockstatus<>'pass'  
                        AND A.CARDCODE ='UM0864'
                order by a.cardcode, a.docdate   
                """

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df = pd.concat(datalist)  

env = Environment(loader=FileSystemLoader('/data/jupyther'))

template = env.get_template("ar_timeout_template.html")            

template_var = {"datas":df.values.tolist()}

html_out = template.render(template_var)

with open("/data/jupyther/locktimeout.html", "w") as fh:
    fh.write(html_out)        


